In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from datetime import datetime

import matplotlib.pyplot as plt
from matplotlib import cm
import numpy as np
import pandas as pd
import seaborn as sns

from bikes.data import load_cycle_counts

## Data Loading

In [ ]:
cycle_counts = load_cycle_counts()
cycle_counts.to_csv("cycle_counts.csv", index=False)

In [ ]:
cycle_counts = pd.read_csv("cycle_counts.csv", parse_dates=["date"])

In [ ]:
from datetime import datetime

from statsmodels.tsa.seasonal import MSTL
from bikes.utils.preprocess import Scaler
from bikes.models.ets import ExponentialSmoothingLearner
from bikes.models.sarima import SARIMASequenceLearner
from bikes.models.prophet import ProphetSequenceLearner
from bikes.models.theta import ThetaSequenceLearner
from bikes.models.prophet import ProphetSequenceLearner
from bikes.models.rnn import RNNSequenceLearner

In [ ]:
i = 5
loc = sorted(cycle_counts["location"].unique())[i]

location_df = cycle_counts[cycle_counts["location"] == loc].sort_values("date").set_index("date")
y_train = location_df.loc[:datetime(2024, 10, 31), "count"]

horizon = 61

print("Fitting ETS model....\n")
ets = ExponentialSmoothingLearner(horizon=horizon, seasonal="add", seasonal_periods=7, freq="D")
ets.fit(y_train)
y_hat_ets = ets.predict()

print("Fitting SARIMA model....\n")
sarima = SARIMASequenceLearner(horizon=horizon, order=(1, 0, 2), seasonal_order=(0, 1, 1, 7), freq="D")
sarima.fit(y_train)
y_hat_sarima = sarima.predict()

print("Fitting Theta model....\n")
theta = ThetaSequenceLearner(horizon=horizon)
theta.fit(y_train)
y_hat_theta = theta.predict()

print("Fitting Prophet model....\n")
prophet = ProphetSequenceLearner(horizon=horizon, growth="linear", changepoint_prior_scale=0.05, yearly_seasonality='auto', weekly_seasonality=3)
prophet.fit(y_train)
y_hat_prophet = prophet.predict()

print("Fitting RNN model....\n")
scaler = Scaler()
y_train_scaled = scaler.fit_transform(y_train)

model = RNNSequenceLearner(horizon=horizon, input_size=1, hidden_size=25)
model.fit(y_train_scaled)
y_hat_rnn = model.predict(y_train_scaled)
y_hat_rnn = pd.Series(scaler.inverse_transform(y_hat_rnn.values), index=y_hat_rnn.index)

In [ ]:
fig, ax = plt.subplots(1, 1, sharex=True, figsize=(12, 2.5))
ax.scatter(
    location_df.index.values[-100:],
    location_df["count"].values[-100:],
    color="black",
    s=15,
)
for name, y_hat in [
    ("ets", y_hat_ets),
    ("sarima", y_hat_sarima),
    ("theta", y_hat_theta),
    ("prophet", y_hat_prophet),
    ("rnn", y_hat_rnn),
]:
    ax.plot(
        y_hat.index.values,
        y_hat.values,
        label=name,
    )
ax.legend()

## EDA

### Total counts by day

In [ ]:
daily_total = cycle_counts.groupby("date")[["count"]].sum().reset_index()

fig, ax = plt.subplots(1, 2, figsize=(10, 3.5))

ax[0].plot(daily_total["date"], daily_total["count"])
ax[0].set(ylabel="Count")
for tick in ax[0].get_xticklabels():
    tick.set_rotation(45)

ax[1].hist(daily_total["count"], bins=20);
ax[1].set(xlabel="Count", ylabel="Frequency")

fig.tight_layout()

### Daily counts by location

In [ ]:
av_daily_counts = cycle_counts.groupby("date")[["count"]].mean().reset_index()

fig, ax = plt.subplots(2, 1, figsize=(12, 5), sharex=True)

locations = cycle_counts["location"].unique()
for location in locations:
    outlier_df = cycle_counts[cycle_counts["location"] == location]
    ax[0].plot(outlier_df["date"], outlier_df["count"], color="gray", alpha=0.25)
    ax[1].plot(outlier_df["date"], np.log1p(outlier_df["count"]), color="gray", alpha=0.25)

ax[0].plot(av_daily_counts["date"], av_daily_counts["count"], color="blue", linewidth=2)
ax[1].plot(av_daily_counts["date"], np.log1p(av_daily_counts["count"]), color="blue", linewidth=2)

ax[0].set(ylabel="Count")
ax[1].set(ylabel="Log(Count)")

In [ ]:
av_daily_counts_by_loc = cycle_counts.groupby("location")[["count"]].mean().reset_index()
av_daily_counts_by_loc = av_daily_counts_by_loc.sort_values("count").reset_index(drop=True)

mean_counts = av_daily_counts_by_loc["count"].mean()
median_counts = av_daily_counts_by_loc["count"].median()

fig, ax = plt.subplots(1, 1)
ax.hist(av_daily_counts_by_loc["count"], bins=20)
ax.axvline(mean_counts, color="red", linestyle="--", lw=2.5, label="Mean")
ax.axvline(median_counts, color="orange", linestyle="--", lw=2.5, label="Median")
ax.set(xlabel="Average Daily Count", ylabel="Frequency")
ax.legend();

### High demand locations

In [ ]:
threshold = 100
high_demand_locs = av_daily_counts_by_loc[av_daily_counts_by_loc["count"] > threshold]
high_demand_locs.sort_values("count")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(10, 5))

locations = high_demand_locs["location"].values
colors = cm.coolwarm(np.linspace(0, 1, len(locations)))
for i, location in enumerate(locations):
    outlier_df = cycle_counts[cycle_counts["location"] == location].sort_values("date").iloc[-150:]
    ax.plot(
        outlier_df["date"],
        outlier_df["count"],
        label=location,
        color=colors[i],
        lw=2.5,
    )
ax.set(ylabel="Count")
fig.tight_layout();

In [ ]:
counts_pivot = cycle_counts.pivot_table(columns="date", index="location", values="count", aggfunc="sum")
counts_pivot = counts_pivot.fillna(0).assign(average=lambda x: x.mean(axis=1))
counts_pivot = counts_pivot[counts_pivot["average"].between(0, 100)]
counts_pivot = counts_pivot.sort_values("average", ascending=False).drop(columns="average")

fig, ax = plt.subplots(1, 1)
sns.heatmap(counts_pivot, cmap="coolwarm", cbar_kws={"label": "Count"})

xs = np.arange(0, counts_pivot.shape[1], 60)
xticklables = [c.strftime("%Y-%m-%d") for c in counts_pivot.columns[xs]]
ax.set(xticks=xs, xticklabels=xticklables)

ys = np.arange(0, counts_pivot.shape[0])
yticklables = counts_pivot.index[ys]
ax.set_yticks(ys + 0.5)
ax.set_yticklabels(yticklables, fontsize="small")

ax.set(xlabel="", ylabel="");